## 1

In [1]:
#url <- "https://raw.githubusercontent.com/qwerty29544/RpracticeBook/master/2Data/01FlatTables/ECG_yurchenkov.txt"
#download_folder <- "./data_input/"
#
#download.file(url, paste(download_folder, "ECG_yurchenkov.txt", sep=""))

## 2

In [2]:
extract_variables <- function(block, vars) {
    result <- c()
    
    index <- 0
    variable <- ''
    for (name in vars) {
        index <- grep(paste('<', name, '>', sep = ''), block)
        variable <- sub(paste(' <', name, '>', sep = ''), '', block[index])
        result <- c(result, variable)
    }
    
    return (result)
}

In [3]:
parse_track_info <- function(strings) {
    variables <- extract_variables(strings, c('Аббревиатура канала', 'Номер трека', 'Тип трека', 
                                              'Единица измерения', 'Частота дискретизации, Гц'))
    track_info <- list(
        abbr = variables[1],
        number = as.integer(variables[2]),
        type = variables[3],
        unit = variables[4],
        frequency = as.numeric(variables[5])
    )
    class(track_info) <- 'TrackInfo'
    
    return (track_info)    
}

In [4]:
parse_fragment <- function(strings) {
    variables <- extract_variables(strings[1:5], c('Название фрагмента', 'Время начала фрагмента от начала исследования, в мс',
                                              'Длина фрагмента, в мс', 'Длина фрагмента, в отсчетах'))
    
    data <- read.table(text = strings[6:length(strings)], sep = "\t", as.is = TRUE)
    colnames(data) <- data[1,]
    data <- data[-1, ]
    rownames(data) <- NULL

    fragment <- list(
        name = variables[1],
        start_time = as.numeric(variables[2]),
        time_length = as.numeric(variables[3]),
        counts_length = as.numeric(variables[4]),
        
        data = data
    )
    class(fragment) <- 'Fragment'
    
    return (fragment)
}

In [5]:
parse_ecg <- function(file_name) {
    all_lines <- readLines(file_name)
    
    variables <- extract_variables(all_lines[1:5], c('Группа', 'Наименование', 'Число треков', 'Число фрагментов'))
    
    tracks_end <- 6
    for (i in 6:length(all_lines)) {
        if (startsWith(all_lines[i], '// Информация по фрагменту')) {
            tracks_end <- i
            break
        }
    }
    tracks_bounds <- c()
    for (i in 6:tracks_end) {
        if (startsWith(all_lines[i], '// Информация о треке')) {
            tracks_bounds <- c(tracks_bounds, i)
        }
    }
    tracks_bounds <- c(tracks_bounds, tracks_end)
    
    tracks_infos <- list()
    for (i in 1:(length(tracks_bounds) - 1)) {
        from <- tracks_bounds[i]
        to <- tracks_bounds[i + 1] - 1
        tracks_infos[[i]] <- parse_track_info(all_lines[from:to])
    }
    
    fragment_bounds <- c()
    for (i in 1:length(all_lines)) {
        if (startsWith(all_lines[i], '// Информация по фрагменту')) {
            fragment_bounds <- c(fragment_bounds, i)
        }
    }
    fragment_bounds <- c(fragment_bounds, length(all_lines))
    
    fragments <- list()
    for (i in 1:(length(fragment_bounds) - 1)) {
        from <- fragment_bounds[i]
        to <- fragment_bounds[i + 1] - 1
        fragments[[i]] <- parse_fragment(all_lines[from:to])
    }
    
    ecg <- list(
        group = variables[1],
        name = variables[2],
        tracks_count = as.integer(variables[3]),
        fragments_count = as.integer(variables[4])
    )
    ecg$tracks <- tracks_infos
    ecg$fragments <- fragments
    
    class(ecg) <- 'EKG'
    
    return (ecg)
}

Как же я заебался парсить эту хрень...

In [6]:
ecg <- parse_ecg('./data_input/ECG_yurchenkov.txt')

In [7]:
head(ecg$fragments[[3]])

Время(мс),0,1,2,3,4
0,"-0,084","0,100625","-1,5625","-0,621033","1,1"
4,"-0,14575","0,095","-1,3","-0,621033","1,075"
8,"-0,15925","0,090625","-1,0375","-0,619507","1,0625"
12,"-0,10025","0,086875","-0,85","-0,621033","1,0375"
16,"-0,06725","0,084375","-0,7375","-0,619507","1,025"
20,"-0,0765","0,08125","-0,6875","-0,621033","1,0125"
24,"-0,0515","0,07875","-0,6875","-0,619507",1
28,"-0,00775","0,07625","-0,7375","-0,619507","0,9875"
32,"-0,0315","0,073125","-0,8","-0,619507","0,9875"
36,"-0,09875","0,07","-0,85","-0,619507","0,975"
